In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = "../datasplit_dataset"

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/train", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

val_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/val", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

test_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/test", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'split_dataset/train'

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # On gèle les couches de ResNet

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Accuracy sur test : {test_acc:.2f}")